In [1]:
metadata = {
    "create_an_account": {
        "sample_payload": {"name": "string"},
        "prerequisite": None
    },
    "buy_product": {
        "sample_payload": {"product": "string"},
        "prerequisite": "create_an_account"
    }
}

In [2]:
event_stream = [
    {"event": "user_message", "content": "I'd like to buy a product."},
    {"event": "api_call", "api_name": "create_an_account", "response": {"status": "success", "user_id": 123}},
    {"event": "user_message", "content": "My name is John Doe."}
]

In [3]:
def create_an_account(name):
    # Simulate the API call to create an account
    print(f"API Call: create_an_account with name={name}")
    # Mock response
    response = {"status": "success", "user_id": 123}
    # Log the response to the event stream
    event_stream.append({"event": "api_call", "api_name": "create_an_account", "response": response})
    return response

def buy_product(product, user_id):
    # Simulate the API call to buy a product
    print(f"API Call: buy_product with product={product} for user_id={user_id}")
    # Mock response
    response = {"status": "success", "order_id": 456}
    # Log the response to the event stream
    event_stream.append({"event": "api_call", "api_name": "buy_product", "response": response})
    return response

In [4]:
def find_in_event_stream(key):
    """
    Helper function to find a piece of information in the event stream.
    """
    for event in reversed(event_stream):  # Reverse to find the most recent occurrence
        if event.get("event") == "user_message" and key.lower() in event.get("content", "").lower():
            return event.get("content")
    return None

def check_prerequisite(api_name):
    """
    Check if the prerequisite API call for the given API has been made.
    """
    prerequisite = metadata[api_name]["prerequisite"]
    if not prerequisite:
        return True
    # Check if the prerequisite API call is in the event stream with a success status
    for event in event_stream:
        if event.get("event") == "api_call" and event.get("api_name") == prerequisite and event["response"].get("status") == "success":
            return True
    return False

def resolve_and_execute(api_name):
    """
    Resolve dependencies and execute the specified API call.
    """
    if not check_prerequisite(api_name):
        print(f"Error: Prerequisite {metadata[api_name]['prerequisite']} not met for {api_name}.")
        return None

    # Resolve the payload for the API call
    payload = {}
    for key in metadata[api_name]["sample_payload"]:
        value = find_in_event_stream(key)
        if not value:
            # Prompt user for missing information
            value = input(f"Please provide {key}: ")
            event_stream.append({"event": "user_message", "content": f"My {key} is {value}."})
        payload[key] = value.split(" is ")[-1]

    # Execute the API call
    if api_name == "create_an_account":
        return create_an_account(payload["name"])
    elif api_name == "buy_product":
        # Check if user_id is available in the event stream
        user_id_event = next((event for event in event_stream if event.get("api_name") == "create_an_account"), None)
        if not user_id_event:
            print("Error: User account not created yet.")
            return None
        user_id = user_id_event["response"]["user_id"]
        return buy_product(payload["product"], user_id)
    else:
        print(f"Unknown API call: {api_name}")
        return None


In [5]:
# Simulate a user interaction where they want to buy a product
print("User wants to buy a product.")

# Agent decides to invoke the "buy_product" API
resolve_and_execute("buy_product")

User wants to buy a product.
API Call: buy_product with product=I'd like to buy a product. for user_id=123


{'status': 'success', 'order_id': 456}

In [6]:
# Output:
# Error: Prerequisite create_an_account not met for buy_product.

# The agent now realizes that it needs to create an account first
resolve_and_execute("create_an_account")

# User provides the necessary information, and the account is created.

# Try buying the product again
resolve_and_execute("buy_product")

API Call: create_an_account with name=John Doe.
API Call: buy_product with product=I'd like to buy a product. for user_id=123


{'status': 'success', 'order_id': 456}

In [7]:
event_stream = [
    {"event": "user_message", "content": "I'd like to buy a product."},
    # No other information provided
]

In [8]:
# Simulate a user interaction where they want to buy a product
print("User wants to buy a product.")

# Agent decides to invoke the "buy_product" API
resolve_and_execute("buy_product")

User wants to buy a product.
Error: Prerequisite create_an_account not met for buy_product.
